In [2]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [3]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [4]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


DataFrames have been loaded successfully.


In [5]:
# Defining small sample vs. end smaple
whole_data_set = False
# end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=420, random_state=RSEED)
    test_df = test_df.sample(n=20, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

<span style="color:red"> this is a small sample : 48692</span>

Calss definition dataloader (do not change over models)

Step 1: U-Net Model and Data Preparation

predictions

In [6]:
train_df

,study_id,severity,condition,level,series_id,x,y,image_path,missing_image
1935,684747623,2.0,0,l5/s1,3.273038e+09,370.053272,506.149133,data/train_images_origin/684747623/774494956/7...,False
6494,2325650566,1.0,0,l5/s1,2.076691e+09,293.720819,472.222032,data/train_images_origin/2325650566/20477869/5...,False
1720,624881903,2.0,0,l5/s1,1.233162e+09,375.776524,475.359848,data/train_images_origin/624881903/1233161684/...,False
9120,3221995449,1.0,0,l5/s1,2.156977e+08,231.017145,348.249377,data/train_images_origin/3221995449/215697714/...,False
360,117720278,0.0,0,l5/s1,4.077719e+09,250.755900,308.758838,data/train_images_origin/117720278/2514759683/...,False
...,...,...,...,...,...,...,...,...,...
5138,1805845915,1.0,0,l5/s1,1.103738e+09,307.068056,511.972077,data/train_images_origin/1805845915/1561472716...,False
5100,1784445928,0.0,0,l5/s1,1.426592e+09,292.571429,367.683516,data/train_images_origin/1784445928/1426592494...,False
4420,1531171418,1.0,0,l5/s1,3.146554e+09,263.693198,333.429812,data/train_images_origin/1531171418/3146554359...,False
4313,1497488178,1.0,0,l5/s1,1.343628e+09,234.787356,343.698883,data/train_images_origin/1497488178/3312070887...,False


test predicion

In [7]:

test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)


In [8]:
print(test_df.to_string(index=False, header=True)) 

  study_id          x          y                                            image_path
2775207739 157.538462 250.197802  data/train_images_origin/2775207739/3249541180/8.dcm
 664153360 260.194392 306.003738  data/train_images_origin/664153360/1076245514/13.dcm
2273432465 234.036660 287.804481   data/train_images_origin/2273432465/114306451/9.dcm
3777149998 151.528158 190.928463  data/train_images_origin/3777149998/3550597941/8.dcm
3967802493 265.649924 385.753425 data/train_images_origin/3967802493/2054070341/11.dcm
2780118855 189.718464 238.157221 data/train_images_origin/2780118855/2064060968/14.dcm
 247968996 256.553304 339.950125   data/train_images_origin/247968996/256361821/14.dcm
 957176622 304.949451 394.690110   data/train_images_origin/957176622/1889022706/1.dcm
4075603869 206.839695 249.160305  data/train_images_origin/4075603869/2921580289/3.dcm
3605654232 163.569959 244.938272    data/train_images_origin/3605654232/26446529/3.dcm
1839242409 314.134420 294.061100 data/train

predicting

In [9]:
test_data = test_df

In [14]:
import numpy as np
import pandas as pd
import torch
import mlflow
from torch.utils.data import DataLoader

# Load the trained model from MLflow
model_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/491281383988954356/344ed7741b8b4cb9bf87844ab1e511af/artifacts/final_model"
model = mlflow.pytorch.load_model(model_path)


# Print the entire model architecture
print(model)
for name, layer in model.named_children():
    print(f"Layer name: {name}")
    print(layer)
for name, param in model.named_parameters():
    print(f"Layer: {name} | Shape: {param.shape}")

    

UNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
Layer name: conv1
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Layer name: conv2
Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Layer: conv1.weight | Shape: torch.Size([64, 3, 3, 3])
Layer: conv1.bias | Shape: torch.Size([64])
Layer: conv2.weight | Shape: torch.Size([2, 64, 3, 3])
Layer: conv2.bias | Shape: torch.Size([2])


ModuleNotFoundError: No module named 'torchinfo'

In [49]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from PIL import Image
import pydicom

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(img_path)
        image_array = dicom_image.pixel_array
        image = Image.fromarray(image_array)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
        y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)

        return image, torch.tensor([x, y]) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [53]:
class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(img_path)
        image_array = dicom_image.pixel_array
        image = Image.fromarray(image_array)

        # Do not convert to RGB, keep it as a single channel
        if self.transform:
            image = self.transform(image)

        x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
        y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)

        return image, torch.tensor([x, y])

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Adjust for grayscale
])


In [57]:

model.eval()  # Set the model to evaluation mode
batch_size = 32
# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Ensure the model is on the right device


predicted_coords = []  # Initialize the list to store predicted coordinates

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        heatmaps = model(images)
        heatmaps = heatmaps.cpu().numpy()

        # Check if heatmaps is 4D (batch_size, channels, height, width)
        if heatmaps.ndim == 4:
            for batch_idx in range(heatmaps.shape[0]):  # Iterate over batch
                heatmap = heatmaps[batch_idx, 0]  # Assuming we have only one channel
                heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap)) if np.max(heatmap) != np.min(heatmap) else heatmap

                if np.max(heatmap) > 0.5:  # Adjust the threshold as needed
                    y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
                    predicted_coords.append((x, y))
                else:
                    predicted_coords.append((np.nan, np.nan))  # Assign NaN if no valid peak
        else:
            print(f"Unexpected heatmap dimensions: {heatmaps.shape}")

# Save the predicted coordinates along with the original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")
 


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[20, 1, 224, 224] to have 3 channels, but got 1 channels instead

In [56]:


predicted_coords = []  # Initialize the list to store predicted coordinates

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        heatmaps = model(images)
        heatmaps = heatmaps.cpu().numpy()

        # Check if heatmaps is 4D (batch_size, channels, height, width)
        if heatmaps.ndim == 4:
            for batch_idx in range(heatmaps.shape[0]):  # Iterate over batch
                heatmap = heatmaps[batch_idx, 0]  # Assuming we have only one channel
                heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap)) if np.max(heatmap) != np.min(heatmap) else heatmap

                if np.max(heatmap) > 0.5:  # Adjust the threshold as needed
                    y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
                    predicted_coords.append((x, y))
                else:
                    predicted_coords.append((np.nan, np.nan))  # Assign NaN if no valid peak
        else:
            print(f"Unexpected heatmap dimensions: {heatmaps.shape}")

# Save the predicted coordinates along with the original test data
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)
test_df_with_preds.to_csv("test_with_predicted_coordinates.csv", index=False)

print("Inference complete. Predicted coordinates saved.")
 

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[20, 1, 224, 224] to have 3 channels, but got 1 channels instead

In [40]:
test_df_with_preds

,study_id,x,y,image_path,x,y
0,4003253,197.100569,289.457306,data/train_images_origin/4003253/702807833/3.dcm,164,1
1,4646740,235.980844,360.313610,data/train_images_origin/4646740/3486248476/2.dcm,221,1
2,4646740,235.980844,360.313610,data/train_images_origin/4646740/3666319702/16...,221,100
3,4646740,235.980844,360.313610,data/train_images_origin/4646740/3486248476/16...,221,87
4,4646740,235.980844,360.313610,data/train_images_origin/4646740/3666319702/10...,221,107
...,...,...,...,...,...,...
9568,4290709089,211.813953,259.534884,data/train_images_origin/4290709089/4237840455...,180,1
9569,4290709089,211.813953,259.534884,data/train_images_origin/4290709089/3274612423...,177,1
9570,4290709089,211.813953,259.534884,data/train_images_origin/4290709089/3274612423...,176,1
9571,4290709089,211.813953,259.534884,data/train_images_origin/4290709089/4237840455...,183,1


In [72]:
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        heatmaps = model(images)
        heatmaps = heatmaps.cpu().numpy()

        for heatmap in heatmaps:
            print(f"Heatmap shape: {heatmap.shape}")
            print(f"Heatmap values: {np.min(heatmap)}, {np.max(heatmap)}")
            if heatmap.ndim == 2:
                y, x = np.unravel_index(np.argmax(heatmap, axis=None), heatmap.shape)
                print(f"Predicted coordinates: x={x}, y={y}")
                predicted_coords.append((x, y))


Heatmap shape: (2, 224, 224)
Heatmap values: -497.9859924316406, 444.58575439453125
Heatmap shape: (2, 224, 224)
Heatmap values: -498.6551208496094, 437.12969970703125
Heatmap shape: (2, 224, 224)
Heatmap values: -498.027587890625, 442.0610046386719
Heatmap shape: (2, 224, 224)
Heatmap values: -498.7286376953125, 436.3865051269531
Heatmap shape: (2, 224, 224)
Heatmap values: -498.0018615722656, 440.8109436035156
Heatmap shape: (2, 224, 224)
Heatmap values: -498.0718994140625, 442.2533264160156
Heatmap shape: (2, 224, 224)
Heatmap values: -498.6551208496094, 438.95013427734375
Heatmap shape: (2, 224, 224)
Heatmap values: -498.6551208496094, 438.7187194824219
Heatmap shape: (2, 224, 224)
Heatmap values: -498.6551208496094, 438.7991027832031
Heatmap shape: (2, 224, 224)
Heatmap values: -126.7032699584961, 444.58575439453125
Heatmap shape: (2, 224, 224)
Heatmap values: -38.956825256347656, 444.58575439453125
Heatmap shape: (2, 224, 224)
Heatmap values: -2.039106607437134, 444.5857543945312

KeyboardInterrupt: 

In [73]:
test_df_with_preds

,study_id,x,y,image_path,x,y
0,2775207739,157.538462,250.197802,data/train_images_origin/2775207739/3249541180...,183,221
1,664153360,260.194392,306.003738,data/train_images_origin/664153360/1076245514/...,158,1
2,2273432465,234.036660,287.804481,data/train_images_origin/2273432465/114306451/...,203,1
3,3777149998,151.528158,190.928463,data/train_images_origin/3777149998/3550597941...,71,1
4,3967802493,265.649924,385.753425,data/train_images_origin/3967802493/2054070341...,2,1
...,...,...,...,...,...,...
415,626906174,282.830769,363.815385,data/train_images_origin/626906174/3362073405/...,185,1
416,3231592574,209.657025,240.132231,data/train_images_origin/3231592574/40966279/3...,184,219
417,332284668,340.733632,503.927515,data/train_images_origin/332284668/1754494354/...,189,1
418,1874721938,152.009078,226.750524,data/train_images_origin/1874721938/66119274/5...,132,221


In [75]:
import numpy as np
import pandas as pd
import torch
import mlflow
from torch.utils.data import DataLoader

# Load the trained model from MLflow
model_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/491281383988954356/344ed7741b8b4cb9bf87844ab1e511af/artifacts/final_model"
model = mlflow.pytorch.load_model(model_path)
model.eval()  # Set the model to evaluation mode

# Prepare the new test dataset (without coordinates) and DataLoader
test_dataset = MRILocalizationDataset(data=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Ensure the model is on the right device

# Inference on new test data
predicted_coords = []
with torch.no_grad():  # Disable gradient computation for inference
    for images, _ in test_loader:  # Unpack images and ignore labels
        images = images.to(device)
        outputs = model(images)  # Get model output
        outputs = outputs.view(outputs.size(0), -1)  # Reshape to (batch_size, 2) for x, y coordinates
        predicted_coords.extend(outputs.cpu().numpy())  # Move to CPU and add to results

# Convert the predicted coordinates to a DataFrame
output_df = pd.DataFrame(predicted_coords, columns=['x', 'y'])

# Combine original test_df with predicted coordinates
test_df_with_preds = pd.concat([test_df.reset_index(drop=True), output_df], axis=1)



ValueError: 2 columns passed, passed data had 100352 columns

In [76]:
test_df_with_preds

,study_id,x,y,image_path,x,y
0,2775207739,157.538462,250.197802,data/train_images_origin/2775207739/3249541180...,183,221
1,664153360,260.194392,306.003738,data/train_images_origin/664153360/1076245514/...,158,1
2,2273432465,234.036660,287.804481,data/train_images_origin/2273432465/114306451/...,203,1
3,3777149998,151.528158,190.928463,data/train_images_origin/3777149998/3550597941...,71,1
4,3967802493,265.649924,385.753425,data/train_images_origin/3967802493/2054070341...,2,1
...,...,...,...,...,...,...
415,626906174,282.830769,363.815385,data/train_images_origin/626906174/3362073405/...,185,1
416,3231592574,209.657025,240.132231,data/train_images_origin/3231592574/40966279/3...,184,219
417,332284668,340.733632,503.927515,data/train_images_origin/332284668/1754494354/...,189,1
418,1874721938,152.009078,226.750524,data/train_images_origin/1874721938/66119274/5...,132,221
